In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Conv1D, Flatten, MaxPooling1D, AveragePooling1D, Concatenate

Using TensorFlow backend.


In [2]:
df = pickle.load(open('dataset_vBalanced_new.pkl', 'rb'))
df.head()

,Target_Vec_0,Target_Vec_1,Target_Vec_2,Target_Vec_3,Target_Vec_4,Target_Vec_5,Target_Vec_6,Target_Vec_7,Target_Vec_8,Target_Vec_9,...,Drug_Vec_91,Drug_Vec_92,Drug_Vec_93,Drug_Vec_94,Drug_Vec_95,Drug_Vec_96,Drug_Vec_97,Drug_Vec_98,Drug_Vec_99,Class
0,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.027773,-0.261224,-0.349108,0.229779,0.161475,0.124695,0.368989,-0.198983,-0.435481,1
1,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.009263,-0.166779,-0.276308,0.178456,0.128912,0.083457,0.357332,-0.184330,-0.500235,1
2,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.009263,-0.166779,-0.276308,0.178456,0.128912,0.083457,0.357332,-0.184330,-0.500235,1
3,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.197189,-0.190077,-0.227381,0.206817,0.102296,0.015936,0.315439,-0.296255,-0.259058,1
4,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.197189,-0.190077,-0.227381,0.206817,0.102296,0.015936,0.315439,-0.296255,-0.259058,1


In [3]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [4]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, train_size=0.8)

## Logistic

In [5]:
lr = LogisticRegression()
lr.fit(train_x, train_y)
print('Score of Logistic :', lr.score(test_x, test_y))
y_pred = lr.predict(test_x)
confusion_matrix(y_pred, test_y)

/home/sarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Score of Logistic : 0.7740102964672466


array([[1547,   34],
       [1239, 2813]])

## Deep NN

In [6]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=200))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

W1019 21:38:21.646733 139647467345280 deprecation_wrapper.py:119] From /home/sarthak/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1019 21:38:21.760612 139647467345280 deprecation_wrapper.py:119] From /home/sarthak/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1019 21:38:21.798738 139647467345280 deprecation_wrapper.py:119] From /home/sarthak/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1019 21:38:21.830993 139647467345280 deprecation_wrapper.py:119] From /home/sarthak/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.place

In [7]:
model.fit(train_x, train_y, epochs=30, batch_size=64)

Epoch 1/30
22529/22529 [==============================] - 4s 183us/step - loss: 0.4806 - acc: 0.7550
Epoch 2/30
22529/22529 [==============================] - 1s 66us/step - loss: 0.4570 - acc: 0.7665
Epoch 3/30
22529/22529 [==============================] - 1s 65us/step - loss: 0.4557 - acc: 0.7675
Epoch 4/30
22529/22529 [==============================] - 1s 65us/step - loss: 0.4526 - acc: 0.7666
Epoch 5/30
22529/22529 [==============================] - 1s 66us/step - loss: 0.4513 - acc: 0.7672
Epoch 6/30
22529/22529 [==============================] - 2s 68us/step - loss: 0.4497 - acc: 0.7673
Epoch 7/30
22529/22529 [==============================] - 2s 89us/step - loss: 0.4479 - acc: 0.7675
Epoch 8/30
22529/22529 [==============================] - 2s 89us/step - loss: 0.4469 - acc: 0.7671
Epoch 9/30
22529/22529 [==============================] - 2s 91us/step - loss: 0.4462 - acc: 0.7668
Epoch 10/30
22529/22529 [==============================] - 2s 89us/step - loss: 0.4449 - acc: 0.767

## Deep Conv NN

In [8]:
ndata, nfeature = X.shape
X_Conv = X.reshape(ndata, nfeature, 1)
input_shape = (nfeature, 1)
ctrain_x, ctest_x, ctrain_y, ctest_y = train_test_split(X_Conv, Y, train_size=0.8)

In [9]:
model = Sequential()

model.add(Conv1D(filters=30, kernel_size=2, input_shape=input_shape,kernel_initializer= 'uniform', activation= 'relu')) 
model.add(Conv1D(filters=12, kernel_size=2, kernel_initializer= 'uniform', activation= 'relu')) 
model.add(Conv1D(filters=10, kernel_size=2, kernel_initializer= 'uniform', activation= 'relu')) 

model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

W1018 16:40:07.932295 140654707774848 deprecation_wrapper.py:119] From /home/sarthak/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [10]:
model.fit(ctrain_x, train_y, epochs=2, batch_size=32)

Epoch 1/2
22529/22529 [==============================] - 16s 720us/step - loss: 0.6932 - acc: 0.5071
Epoch 2/2
22529/22529 [==============================] - 23s 1ms/step - loss: 0.6934 - acc: 0.5002


## Deep Conv NN with seperate conv for target and drug

In [11]:
target = df.iloc[:, :100].values
drug = df.iloc[:, 100:-1].values
ndata, nfeature = target.shape
input_shape = (nfeature, 1)
train_idx, test_idx, dt_train_y, dt_test_y = train_test_split(np.arange(0, ndata), Y, train_size=0.8)
train_target = target[train_idx]
test_target = target[test_idx]
train_drug = drug[train_idx]
test_drug = drug[test_idx]

In [12]:
train_size = train_target.shape[0]
test_size = test_target.shape[0]
train_target = train_target.reshape(train_size, nfeature, 1)
test_target = test_target.reshape(test_size, nfeature, 1)
train_drug = train_target.reshape(train_size, nfeature, 1)
test_drug = test_target.reshape(test_size, nfeature, 1)

In [13]:
model1 = Sequential(layers=[
    # input layers and convolutional layers
    Conv1D(filters=30, kernel_size=2, input_shape=input_shape,kernel_initializer= 'uniform', activation= 'relu'),
    Conv1D(filters=10, kernel_size=2, kernel_initializer= 'uniform', activation= 'relu'),
    Dropout(.5),

])

model2 = Sequential(layers=[
    # input layers and convolutional layers
    Conv1D(filters=30, kernel_size=2, input_shape=input_shape,kernel_initializer= 'uniform', activation= 'relu'),
    Conv1D(filters=10, kernel_size=2, kernel_initializer= 'uniform', activation= 'relu'),
    Dropout(.5),

])

mergedOut = Concatenate()([model1.output,model2.output])
mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(256, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(128, activation='relu')(mergedOut)
mergedOut = Dropout(.35)(mergedOut)

# output layer
mergedOut = Dense(1, activation='sigmoid')(mergedOut)
newModel = Model([model1.input,model2.input], mergedOut)

newModel.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
newModel.fit([train_target, train_drug], dt_train_y,epochs=2, batch_size=32)   

Epoch 1/2
22529/22529 [==============================] - 33s 1ms/step - loss: 0.5480 - acc: 0.6834
Epoch 2/2
22529/22529 [==============================] - 28s 1ms/step - loss: 0.5425 - acc: 0.6880
